# Ajuste fino de modelos de Open AI

Este cuaderno se basa en la guía actual proporcionada en la documentación de [Ajuste Fino](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) de Open AI.

El ajuste fino mejora el rendimiento de los modelos base para tu aplicación al reentrenarlos con datos y contexto adicionales relevantes para ese caso de uso o escenario específico. Ten en cuenta que técnicas de ingeniería de prompts como _few shot learning_ y _retrieval augmented generation_ te permiten mejorar el prompt predeterminado con datos relevantes para aumentar la calidad. Sin embargo, estos enfoques están limitados por el tamaño máximo de la ventana de tokens del modelo base seleccionado.

Con el ajuste fino, en realidad estamos reentrenando el propio modelo con los datos necesarios (lo que nos permite usar muchos más ejemplos de los que caben en la ventana máxima de tokens) y desplegando una versión _personalizada_ del modelo que ya no necesita que se le proporcionen ejemplos en tiempo de inferencia. Esto no solo mejora la efectividad del diseño de nuestros prompts (tenemos más flexibilidad para usar la ventana de tokens en otras cosas), sino que también puede reducir nuestros costos (al disminuir la cantidad de tokens que necesitamos enviar al modelo durante la inferencia).

El ajuste fino tiene 4 pasos:
1. Preparar los datos de entrenamiento y subirlos.
1. Ejecutar el trabajo de entrenamiento para obtener un modelo ajustado.
1. Evaluar el modelo ajustado y repetir para mejorar la calidad.
1. Desplegar el modelo ajustado para inferencia cuando estés satisfecho.

Ten en cuenta que no todos los modelos base admiten ajuste fino; [consulta la documentación de OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) para obtener la información más reciente. También puedes ajustar un modelo que ya ha sido ajustado previamente. En este tutorial, usaremos `gpt-35-turbo` como nuestro modelo base objetivo para el ajuste fino.

---


### Paso 1.1: Prepara tu conjunto de datos

Vamos a crear un chatbot que te ayude a entender la tabla periódica de los elementos respondiendo preguntas sobre un elemento con una limerick. En _este_ sencillo tutorial, solo crearemos un conjunto de datos para entrenar el modelo con algunos ejemplos de respuestas que muestran el formato esperado de los datos. En un caso de uso real, necesitarías crear un conjunto de datos con muchos más ejemplos. También podrías usar un conjunto de datos abierto (para tu dominio de aplicación) si existe alguno, y reformatearlo para usarlo en el ajuste fino.

Como nos estamos enfocando en `gpt-35-turbo` y buscamos una respuesta de un solo turno (chat completion), podemos crear ejemplos usando [este formato sugerido](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) que refleja los requisitos de chat completion de OpenAI. Si esperas contenido conversacional de varios turnos, deberías usar el [formato de ejemplo de varios turnos](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), que incluye un parámetro `weight` para indicar qué mensajes deben usarse (o no) en el proceso de ajuste fino.

Usaremos el formato más sencillo de un solo turno para nuestro tutorial aquí. Los datos están en el [formato jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) con 1 registro por línea, cada uno representado como un objeto en formato JSON. El fragmento a continuación muestra 2 registros como ejemplo; consulta [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) para ver el conjunto completo de ejemplos (10 ejemplos) que usaremos en nuestro tutorial de ajuste fino. **Nota:** Cada registro _debe_ estar definido en una sola línea (no dividido en varias líneas como es típico en un archivo JSON formateado)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

En un caso de uso real necesitarás un conjunto de ejemplos mucho más grande para obtener buenos resultados; el equilibrio estará entre la calidad de las respuestas y el tiempo/costo del ajuste fino. Estamos usando un conjunto pequeño para poder completar el ajuste fino rápidamente y así ilustrar el proceso. Consulta [este ejemplo del OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) para un tutorial de ajuste fino más complejo.


### Paso 1.2 Sube tu conjunto de datos

Sube los datos usando la API de Archivos [como se describe aquí](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Ten en cuenta que para ejecutar este código, primero debes haber realizado los siguientes pasos:
 - Haber instalado el paquete de Python `openai` (asegúrate de usar una versión >=0.28.0 para contar con las funciones más recientes)
 - Haber configurado la variable de entorno `OPENAI_API_KEY` con tu clave de API de OpenAI
Para más información, consulta la [Guía de configuración](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) proporcionada para el curso.

Ahora, ejecuta el código para crear un archivo a partir de tu archivo JSONL local y subirlo.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Paso 2.1: Crea el trabajo de ajuste fino con el SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Paso 2.2: Verificar el estado del trabajo

Aquí tienes algunas cosas que puedes hacer con la API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Lista los últimos n trabajos de ajuste fino
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Obtiene los detalles de un trabajo de ajuste fino específico
- `client.fine_tuning.jobs.cancel(<job_id>)` - Cancela un trabajo de ajuste fino
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Muestra hasta n eventos del trabajo
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

El primer paso del proceso es _validar el archivo de entrenamiento_ para asegurarse de que los datos estén en el formato correcto.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Paso 2.3: Haz seguimiento de eventos para monitorear el progreso


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Paso 2.4: Ver el estado en el panel de OpenAI


También puedes ver el estado visitando el sitio web de OpenAI y explorando la sección _Fine-tuning_ de la plataforma. Allí podrás ver el estado del trabajo actual y también seguir el historial de ejecuciones anteriores. En esta captura de pantalla, puedes notar que la ejecución previa falló y la segunda se realizó con éxito. Para dar contexto, esto ocurrió porque la primera ejecución utilizó un archivo JSON con registros mal formateados; una vez corregido, la segunda ejecución terminó correctamente y el modelo quedó disponible para su uso.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.es.png)


También puedes ver los mensajes de estado y las métricas desplazándote hacia abajo en el panel visual, como se muestra:

| Mensajes | Métricas |
|:---|:---|
| ![Mensajes](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.es.png) |  ![Métricas](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.es.png)|


### Paso 3.1: Recuperar ID y probar el modelo ajustado en el código


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Paso 3.2: Cargar y probar el modelo ajustado en Playground

Ahora puedes probar el modelo ajustado de dos maneras. Primero, puedes ir a Playground y usar el menú desplegable de Modelos para seleccionar tu modelo ajustado de la lista de opciones. La otra opción es usar la opción "Playground" que aparece en el panel de Fine-tuning (ver la captura de pantalla arriba), la cual abre una vista _comparativa_ que muestra las versiones del modelo base y el modelo ajustado lado a lado para una evaluación rápida.

![Estado del trabajo de fine-tuning](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.es.png)

Simplemente completa el contexto del sistema que usaste en tus datos de entrenamiento y proporciona tu pregunta de prueba. Notarás que ambos lados se actualizan con el mismo contexto y pregunta. Ejecuta la comparación y verás la diferencia en las respuestas entre ambos modelos. _Observa cómo el modelo ajustado genera la respuesta en el formato que diste en tus ejemplos, mientras que el modelo base simplemente sigue la indicación del sistema_.

![Estado del trabajo de fine-tuning](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.es.png)

También notarás que la comparación muestra la cantidad de tokens para cada modelo y el tiempo que tarda la inferencia. **Este ejemplo específico es muy simple y solo busca mostrar el proceso, pero no refleja un conjunto de datos o escenario real**. Puede que notes que ambos ejemplos muestran la misma cantidad de tokens (el contexto del sistema y la indicación del usuario son idénticos), pero el modelo ajustado tarda más en la inferencia (modelo personalizado).

En escenarios reales, no usarás un ejemplo tan simple como este, sino que harás fine-tuning con datos reales (por ejemplo, un catálogo de productos para atención al cliente), donde la calidad de la respuesta será mucho más evidente. En _ese_ contexto, lograr una calidad de respuesta equivalente con el modelo base requerirá una mayor personalización de los prompts, lo que aumentará el uso de tokens y posiblemente el tiempo de procesamiento para la inferencia. _Para probar esto, revisa los ejemplos de fine-tuning en el OpenAI Cookbook para comenzar._



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o inexactitudes. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que surjan del uso de esta traducción.
